In [ ]:
!pip install sympy numpy pandas matplotlib scipy ipython
# this should take up to 7 minutes

In [1]:
from sympy import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fsolve
from IPython.display import display, Math, Latex

In [ ]:
def crate_symbol(name):
    return symbols(name, real=True, positive=True)

        
u = crate_symbol("u")

x1 = crate_symbol("x1")
x2 = crate_symbol("x2")
x3 = crate_symbol("x3")
x4 = crate_symbol("x4")
x5 = crate_symbol("x5")

a1 = crate_symbol("a1")
a2 = crate_symbol("a2")
a3 = crate_symbol("a3")
a4 = crate_symbol("a4")
a5 = crate_symbol("a5")

b1 = crate_symbol("b1")
b2 = crate_symbol("b2")
b3 = crate_symbol("b3")
b4 = crate_symbol("b4")
b5 = crate_symbol("b5")

K_GR = crate_symbol("K_GR")
H1 = crate_symbol("H1")
H2 = crate_symbol("H2")
P = crate_symbol("P")
A = crate_symbol("A")
t = crate_symbol("t")

aP = crate_symbol("aP")
aA = crate_symbol("aA")

bP = crate_symbol("bP")
bA = crate_symbol("bA")


mr = 1 / x3
gr = K_GR ** 3 / (x3 ** 3 + K_GR ** 3)
# # gr = 1 / (1 + (x_3 / K_GR) ** 3)
# gr = 1
f = gr * mr
dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
dx2_dt = b2 * x1 * P * gr * x5- a2 * x2
dx3_dt = b3 * x2 * A *x4 * x5 - a3 * x3
dx4_dt = b4 * u -  a4 * x4
dx5_dt = b5 * u * gr - a5 * x5
dP_dt = P * (bP * x1 * x4 - aP)
dA_dt = A * (bA * x2 - aA)

display(
    Matrix(
        [
            Eq(Derivative(x1, t), dx1_dt),
            Eq(Derivative(x2, t), dx2_dt),
            Eq(Derivative(x3, t), dx3_dt),
            Eq(Derivative(x4, t), dx4_dt),
            Eq(Derivative(x5, t), dx5_dt),
        ]
    )
)
model = Matrix([dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt])
variables = (x1, x2, x3, x4, x5, P, A)

In [3]:
parameters = {
    u: 1,
    a1: 1,
    a2: 1,
    a3: 1,
    a4: 1,
    a5: 1,
    b1: 1,
    b2: 1,
    b3: 1,
    b4: 1,
    b5: 1,
    K_GR: 4,
    H1: 1,
    H2: 1,
    aA: 1,
    aP: 1,
    bA: 1,
    bP: 1,
    # P: 1,
    # A: 1,
    # x1: 1,
    # x2: 1,
    # x3: 1,
    # x4: 1,
    # x5: 1,
}

In [ ]:
parameter_values_normal = {
    u_ext: 1,
    b_u: 0.17 * 60 * 24 * 480,
    b_1: 0.17 * 60 * 24,
    b_2: 0.035 * 60 * 24,
    b_3: 0.0086 * 60 * 24,
    b_P: 0.049,
    b_A: 0.099,
    a_u: 0.17 * 60 * 24 * 480,
    a_1: 0.17 * 60 * 24,
    a_2: 0.035 * 60 * 24,
    a_3: 0.0086 * 60 * 24,
    a_P: 0.049,
    a_A: 0.099,
    a_kgr: 2,
    b_kgr: 2,
    p_kgr: 9,
    u_T: 5,
    a_M: 0.05,
    b_M: 0.05,
    kgr_0: 5,
    t: 1,
    p_M: 50,
    x_2_ext:0,
    K_GR: K_GR_function,
}
def model_simulator(y, t, model_dictionary):
    for time, cur_model in model_dictionary.items():
        if time[0] <= t < time[1]:
            return cur_model['model'](*y).flatten()


def model_fixed_point_function(ic, model_function):
    return model_function(*ic).flatten()


def simulate_HPA(ic, sim_time, model_dictionary, variables):
    result = odeint(model_simulator, ic, sim_time, args=(model_dictionary,), hmax=0.01)
    result_df = pd.DataFrame(result, columns=variables)
    return result_df

model_dictionary = {  # specify which model to use in each time interval 
    # (10 * day, 90 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (10 * day, 30 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (10 * day, 100 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (14 * day, 16 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (18 * day, 20 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (22 * day, 24 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (26 * day, 28 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # 
    # (10 * hour, 13 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (20 * hour, 23 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (30 * hour, 33 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (40 * hour, 43 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (50 * hour, 1000 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    (-np.inf , 30* day): {'model': model_low_u, 'params': parameter_values_sim_low},

    (90* day, 300 *day): {'model': model_low_u, 'params': parameter_values_sim_low},
    # (300 * day, 360 * day): {'model': model_high_x_2, 'params': parameter_values_sim_low_high_x_2},
    # (200 * day, np.inf): {'model': model_high_x_2, 'params': parameter_values_sim_low_high_x_2},"""  """

    (300* day, np.inf): {'model': model_low_u, 'params': parameter_values_sim_low},
    
    (-np.inf, np.inf): {'model': model_high_u, 'params': parameter_values_sim_high}
}

In [ ]:
# time_unit: days
# parameter_values_normal[u_T] = 5

parameter_values_normal[kgr_0] = 2
parameter_values_normal[u_T] = 3
parameter_values_normal[t] = 0.8
# parameter_values_ones[K_GR] = Piecewise((2, M > 5), (4, True)) 
parameter_values_sim_high = parameter_values_normal.copy()
parameter_values_sim_low = parameter_values_normal.copy()
parameter_values_sim_low_high_x_2 = parameter_values_normal.copy()

parameter_values_sim_high[u_ext] = 20
parameter_values_sim_low[u_ext] = 1
parameter_values_sim_low_high_x_2[u_ext] = 1
parameter_values_sim_low_high_x_2[x_2_ext] = 10


model_high_u = lambdify(variables, model.subs(parameter_values_sim_high))
model_low_u = lambdify(variables, model.subs(parameter_values_sim_low))
model_high_x_2= lambdify(variables, model.subs(parameter_values_sim_low_high_x_2))

ic = fsolve(model_fixed_point_function, guess, args=model_low_u)

day = 1
hour = day / 24
year = 365 * day
L = 500 * day
dt = hour
sim_time = np.arange(0, L, dt)

# model_dictionary = {  # specify which model to use in each time interval 
#     (5 * hour , 10  * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
#     (-np.inf, np.inf): {'model': model_low_u, 'params': parameter_values_sim_low}
# }
model_dictionary = {  # specify which model to use in each time interval 
    # (10 * day, 90 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (10 * day, 30 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (10 * day, 100 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (14 * day, 16 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (18 * day, 20 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (22 * day, 24 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (26 * day, 28 * day): {'model': model_high_u, 'params': parameter_values_sim_high},
    # 
    # (10 * hour, 13 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (20 * hour, 23 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (30 * hour, 33 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (40 * hour, 43 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    # (50 * hour, 1000 * hour): {'model': model_high_u, 'params': parameter_values_sim_high},
    (-np.inf , 30* day): {'model': model_low_u, 'params': parameter_values_sim_low},

    (90* day, 300 *day): {'model': model_low_u, 'params': parameter_values_sim_low},
    # (300 * day, 360 * day): {'model': model_high_x_2, 'params': parameter_values_sim_low_high_x_2},
    # (200 * day, np.inf): {'model': model_high_x_2, 'params': parameter_values_sim_low_high_x_2},"""  """

    (300* day, np.inf): {'model': model_low_u, 'params': parameter_values_sim_low},
    
    (-np.inf, np.inf): {'model': model_high_u, 'params': parameter_values_sim_high}
}

# ic = fsolve(model_fixed_point_function, guess, args=model_low_u)
# ic = guess

res = simulate_HPA(ic, sim_time, model_dictionary,variables)
time_scale = 'days'
if time_scale == 'hours':
    res['t'] = sim_time * 24
else:
    res['t'] = sim_time

res = res.set_index('t')
# res.set_index('t', inplace=True)
# display(res)
kgr_lambda = lambdify(M, K_GR.subs(parameter_values_normal))
res['KGR'] = kgr_lambda(res[M])

res.plot(subplots=True, layout=(4, 2), figsize=(10, 10), title='HPA Model', grid=True, sharey=False,
         xlabel=f'Time ({time_scale})', ylabel='Concentration (nM)')
plt.tight_layout()
# plt.xticks(np.arange(0, L, day), np.arange(0, L, 2 * day))
plt.show()

In [ ]:
def run_sim(model, variables,parameters, time, high_u_times=[]):
    model_lambda = lambdify(variables, model.subs(parameters))
    if u in parameters:
        parameters.pop(u)
    def model_ode(y, t):
        for high_u_time in high_u_times:
            if t > high_u_time[0] and t < high_u_time[1]:
                parameters[u] = 2
            else:
                parameters[u] = 1
        return model_lambda(*y).flatten()
    y0 = [1 for _ in variables]
    y = odeint(model_ode, y0, time)
    y_df = pd.DataFrame(y, columns=[str(v) for v in variables])
    return y_df
t = np.linspace(0, 100, 1000)
variables = (u,x1, x2, x3, x4, x5, P, A)
solution  = run_sim(model, variables, parameters, t)
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
# plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()

    

In [ ]:
solution.plot(backend="plotly")

In [ ]:
lambdify(variables, model.subs(parameters)) (*([1] * 7 ))

In [ ]:
model

In [ ]:
# Define the system of differential equations with piecewise function for u
def model_system(y, t, params):
    x1, x2, x3, x4, x5, P,A = y
    a1, a2, a3, a4, a5, b1, b2, b3, b4, b5, K_GR = params
    # print(params)
    # Define u as a piecewise function
    u = 1 if t < 15 or (t > 20 and t < 35) or t > 40 else 2
    K_GR = 4 if t < 30 else 2
    # gr.subs({K_GR:4})
    
    # u = 1 if t < 15 and t <20 else 2
    
    mr = 1 / x3
    gr = 1 / (1 + (x3 / K_GR) ** 3)
    f = gr * mr
    
    dx1_dt = b1 * u * gr * mr - a1 * x1
    dx2_dt = b2 * x1 * P * gr - a2 * x2
    dx3_dt = b3 * x2 * A * x4  - a3 * x3
    dx4_dt = b4 * u * gr - a4 * x4
    dx5_dt = b5 * u * gr - a5 * x5
    dP_dt = P*(x1 * x4 - 1)
    dA_dt = A* (x2  - 1)
    

    # dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
    # dx2_dt = b2 * x1 * P * gr * x5 - a2 * x2
    # dx3_dt = b3 * x2 * A * x4 * x5 * gr - a3 * x3
    # dx4_dt = b4 * u * gr - a4 * x4
    # dx5_dt = b5 * u * gr - a5 * x5
    # print(dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt)
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt]

# Initial conditions
y0 = [1, 1, 1, 1, 1,1,1]

# Time points where solution is computed
t = np.linspace(0, 60, 400)

# Parameters
params = [parameters[a1], parameters[a2], parameters[a3], parameters[a4], parameters[a5],
          parameters[b1], parameters[b2], parameters[b3], parameters[b4], parameters[b5], parameters[K_GR]]

# Solve the system of differential equations
solution = odeint(model_system, y0, t, args=(params,))

# Plot the results
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
# plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()
plt.title('Simulation of Stress with u changing at t=5')
plt.show()

In [ ]:
# Define the system of differential equations with piecewise function for u
def model_system(y, t, params):
    x1, x2, x3, x4, x5, P,A = y
    a1, a2, a3, a4, a5, b1, b2, b3, b4, b5, K_GR = params
    
    # Define u as a piecewise function
    u = 1 if t < 15 or (t > 20 and t < 35) or t > 40 else 2
    K_GR = 4 if t < 30 else 2
    
    # u = 1 if t < 15 and t <20 else 2
    
    mr = 1 / x3
    # gr = 1 / (1 + (x3 / K_GR) ** 3)
    f = gr * mr
    
    dx1_dt = b1 * u * gr * mr - a1 * x1
    dx2_dt = b2 * x1 * P * gr - a2 * x2
    dx3_dt = b3 * x2 * A * x4  - a3 * x3
    dx4_dt = b4 * u * gr - a4 * x4
    dx5_dt = b5 * u * gr - a5 * x5
    dP_dt = P(x1 * x_4 - 1)
    dA_dt = A(x2  - 1)
    

    # dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
    # dx2_dt = b2 * x1 * P * gr * x5 - a2 * x2
    # dx3_dt = b3 * x2 * A * x4 * x5 * gr - a3 * x3
    # dx4_dt = b4 * u * gr - a4 * x4
    # dx5_dt = b5 * u * gr - a5 * x5
    
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt]

# Initial conditions
y0 = [1, 1, 1, 1, 1,1,1]

# Time points where solution is computed
t = np.linspace(0, 60, 400)

# Parameters
params = [parameters[a1], parameters[a2], parameters[a3], parameters[a4], parameters[a5],
          parameters[b1], parameters[b2], parameters[b3], parameters[b4], parameters[b5],
          parameters[K_GR]]

# Solve the system of differential equations
solution = odeint(model_system, y0, t, args=(params,))

# Plot the results
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()
plt.title('Simulation of Stress with u changing at t=5')
plt.show()

In [ ]:
model.subs(parameters)

In [ ]:
model

In [ ]:
def model_fixed_point_function(ic, model_function):
    return model_function(*ic).flatten()

guess = [
    1,  # u
    1,  # x_1
    1,  # x_2
    1,  # x_3
    1,  # P
]
# use lsoda to solve the system
parameters[u] = 1
parameters[x5] = 1
fix_point = solve(model.subs(parameters)[:2], variables)
for point in fix_point:
    for key in point:
        display(key)
# model_f = lambdify(variables, model.subs(parameters))
# ic = fsolve(model_fixed_point_function, guess, args=model_f)
# ic